<A HREF="https://news.gallup.com/poll/116479/barack-obama-presidential-job-approval.aspx">Gallup Approval Numbers</A><BR>

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import time # avoid swamping the americanrhetoric website
import os # to check if directory exists and create it if it doesn't
from datetime import datetime # to parse speech date

In [60]:
URL = 'https://news.gallup.com/poll/116479/barack-obama-presidential-job-approval.aspx'
Page = requests.get(URL, headers={'user-agent': 'Mozilla/5.0'})
soup = BeautifulSoup(Page.content, 'html.parser')

<A HREF="https://regexr.com/">RegExr: for testing regex</A><BR>
    There is also <A HREF="https://regex101.com/">Regex101</A>

In [61]:
# Check if Data and associated folders exist for saving speech CSVs to.
# https://djangocentral.com/check-if-a-directory-exists-if-not-create-it/

MYDIRS = ["approvalData"]

# If folder doesn't exist, then create it.
for MYDIR in MYDIRS:
    CHECK_FOLDER = os.path.isdir(MYDIR)
    if CHECK_FOLDER:
        print(MYDIR, "folder already exists.")
    else:
        os.makedirs(MYDIR)
        print("created folder : ", MYDIR)


approvalData folder already exists.


<A HREF="https://beautiful-soup-4.readthedocs.io/en/latest/">Beautifulsoup documentation</A><BR>
<A HREF="https://docs.python.org/3/library/re.html">Python regex documentation</A>

In [62]:
tables = soup.find_all('table', {'aria-describedby':'table'})

In [78]:
#<tr>
#<th scope="row">2017 Jan 16-19</th>
#<td data-th="Approve" data-thunit="%">59</td>
#<td data-th="Disapprove" data-thunit="%">37</td>
#<td data-th="No opinion" data-thunit="%">4</td>
#</tr>

# dates also grabs the party breakdown numbers, thus list is twice as long as the other three
dates = [[row.text for row in table.find_all('th', {'scope':'row'})] for table in tables if len(table.find_all('th', {'scope':'row'}))>0]
approval = [[row.text for row in table.find_all('td', {'data-th':'Approve'})] for table in tables if len(table.find_all('td', {'data-th':'Approve'}))>0]
disapproval = [[row.text for row in table.find_all('td', {'data-th':'Disapprove'})] for table in tables if len(table.find_all('td', {'data-th':'Disapprove'}))>0]
noOp = [[row.text for row in table.find_all('td', {'data-th':'No opinion'})] for table in tables if len(table.find_all('td', {'data-th':'No opinion'}))>0]



In [80]:
# Flatten the lists into vectors
dates = sum(dates, [])
approval = sum(approval, [])
disapproval = sum(disapproval, [])
noOp=sum(noOp, [])
# use only first half of date data
dates=dates[0:418]

In [81]:
# Parse dates
dates = [date.split('-') for date in dates ]

In [122]:
fulldates=[]
for i in range(len(dates)):
    if len(dates[i][1])<=2:
        fulldates.append([dates[i][0], dates[i][0][0:9]+dates[i][1]])
    elif len(dates[i][1])<=6:
        fulldates.append([dates[i][0], dates[i][0][0:5]+dates[i][1]])
    else:
        fulldates.append([dates[i][0], dates[i][1]])

In [123]:
len(fulldates)==len(approval)

True

<A HREF="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Pandas cheat sheet</A>

In [124]:
beg =[fulldates[i][0] for i in range(len(fulldates))]
end =[fulldates[i][1] for i in range(len(fulldates))]
end = [ date.replace('Sept', 'Sep') for date in end ]

In [125]:
df = pd.DataFrame(
    {"beg": beg,
     "end": end,
     "approval": approval,
     "disapproval": disapproval,
     "no opinon": noOp})

In [126]:
datetime.strptime(df['beg'][34], '%Y %b %d')

datetime.datetime(2016, 5, 23, 0, 0)

<A HREF="https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior">Python Datetime doc</A>

In [127]:
df['beg'] = [ datetime.strptime(date, '%Y %b %d')for date in df['beg'] ]

In [128]:

df['end'] = [ datetime.strptime(date, '%Y %b %d')for date in df['end'] ]

In [132]:
df.to_csv('./Data/genData/approvalData.csv', index=False)

In [147]:
# Just checking probably delete this once we're happy with the data
text1 = '25 October 2005, Washington, D.C.'
text2 = '4 April 1997'
text1.split(',')[0]
text2.split(',')[1:]
date1 = datetime.strptime(text1.split(',')[0], '%d %B %Y')
print(date1)

2005-10-25 00:00:00


In [148]:
date1.year

2005

In [149]:
date1.month

10

In [150]:
date1.day

25